In [1]:
import matplotlib
import os
import inspect
import requests
import time
from tqdm.notebook import tqdm
import random
import pandas as pnd
from boardgamegeek import BoardGameGeek

# Step 1

Access BBG API's to download game and user preferences data.
Define criterion used for setting the parameters that drive the download in termis of the variables defining the dataset, such collecting games only for a specified period or a specified set of categories. The wider the game collection, the better.

#### Dataframe reading

In [2]:
df = pnd.read_csv("BGG_latest.csv")
df.shape

(18931, 9)

In [3]:
#df.sort_values(by=["Rank"]).head()

In [5]:
df = df.loc[df["Year"].isin(range(2000, 2020))]
for col in df.columns:
    print(col)
print("\nShape: ", df.shape)
df.loc[df["ID"] == 34084]
#df_sample = df.sample(frac=0.6).reset_index(drop=True)
df_sample = df

ID
Name
Year
Rank
Average
Bayes average
Users rated
URL
Thumbnail

Shape:  (14115, 9)


#### Construction of empty dataframe with just the important columns

In [6]:
bgg = BoardGameGeek()
g = bgg.game(name=None, game_id=34084)

In [7]:
cols = list(g.__dict__["_data"])
new_df = pnd.DataFrame(columns = cols)
new_df = new_df.drop(["thumbnail", "image", "expansion", "implementations", "expansions", "expands", 
             "alternative_names", "trading", "wanting", "wishing", "numcomments"], axis=1)
new_df

,id,families,categories,mechanics,designers,artists,publishers,description,yearpublished,minplayers,...,name,usersrated,average,bayesaverage,stddev,median,owned,numweights,averageweight,ranks


In [8]:
error_row = []

#### Retrieval of row informations via API requests
Parsing of original filtered dataframe to fill out the new dataframe. <br>
It will be saved as .csv in order not to be recomputed at each iteration.

In [ ]:
for index, row in tqdm(list(df_sample.iterrows())):
    try:
        g = bgg.game(name=None, game_id=row["ID"])
        new_row = {}
        for k, v in g.__dict__["_data"].items():
            if k in new_df.columns:
                #print("{0}: {1}".format(k, v))
                new_row[k] = v
        new_df = new_df.append(new_row, ignore_index=True)
    except:
        print("Error on iteration row {0}".format(index))
        error_row.append(df_sample.iloc[index])
        pass
    time.sleep(0.5)


Error on iteration row 30


In [16]:
while len(error_row) > 0:
    try:
        g = bgg.game(name=None, game_id=error_row[i])
        new_row = {}
        for k, v in g.__dict__["_data"].items():
            if k in new_df.columns:
                new_row[k] = v
        new_df = new_df.append(new_row, ignore_index=True)
        error_row.pop(0)
    except:
        #print("Error on iteration row {0}".format(index))
        error_row.append(df_sample.iloc[index])
        pass


Error on iteration row 14119


IndexError: single positional indexer is out-of-bounds

In [13]:
new_df.drop_duplicates(subset="id", keep=False, inplace=True)
new_df.shape

(9684, 23)

In [22]:
#print(error_row)
new_df.to_csv("./BGG_sampled.csv")
#new_df.head()

# TODO
### Sampled dataframe, exported as csv

In [ ]:
list(df.iterrows())[:3]

In [24]:
stub = [(index, bgg.game(name=None, game_id=row["ID"])) for index, row in df_sample.iterrows()]

KeyboardInterrupt: 

In [ ]:
g.__dict__

### The CSV is solid and sound

All games between the years 2000 and 2020 are considered

for i in range(1, 7):
    g = bgg.game(name=None, game_id=i)
    infos = """Name: {0} \nRating Average: {1} \nRating Average Weight: {2}
    \nRating Bayes Average: {3} \nRating Std Dev: {4} \nNumber of Weights: {5} 
    \nMax Players: {6} \nMin Players: {7} \nMin Age: {8} \nPlaying Time: {9} \nYear: {10}
    \nUsers Commented: {11} \nUsers Owned: {12} \nUsers Rated: {13} \nUsers Trading: {14}
    \nUsers Wanting: {15} \nUsers Wishing: {16} \nCategories: {17} \nDesigners: {18} \nFamilies: {19} \nPlaying Time: {20}\n
    """.format(g.name, g.rating_average, g.rating_average_weight, \
            g.rating_bayes_average, g.rating_stddev, g.rating_num_weights, \
            g.max_players, g.min_players, g.min_age, g.playing_time, g.year, \
            g.users_commented, g.users_owned, g.users_rated, g.users_trading, \
            g.users_wanting, g.users_wishing, (', '.join(g.categories)), (', '.join(g.designers)), \
            (', '.join(g.families)), g.playing_time)
    print(infos)

## Step 2
Pre-process game text (title and decription) in order to define the terms to include in the variables to be observed, eventually using a word embedding model to normalize and/or expand the teminology extracted from the game description


## Step 3
Perform a correlation analysis of the variables. On top of this, try to design and implement a model for predicting the "subjective" variable values starting from the "objective" variables. Optionally, define a recommendation system for games.

## Step 4
Include in the paper a report describing the output of Step 3